In [1]:
import os
import json
import logging
import sys

from datetime import datetime, timedelta

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import xarray as xr
from netCDF4 import Dataset

from scipy.spatial import KDTree
import folium

In [2]:
with open('../config.json', 'r') as config_file:
    config = json.load(config_file)
    api_key = config["meteo_api"]

save_dir = os.path.join(config["data_path"], "meteo_raw")
data_dir = config["data_path"]

# Import KPI dataset

In [3]:
# Load BTS Spatial Data to get the list of relevant weather stations
bts_spatial = pd.read_csv(os.path.join(data_dir, "spatial_new.csv"))
print(bts_spatial.head())

# Get unique station IDs needed
unique_stations = set(bts_spatial["nearest_station"].astype(str))
print(f"Number of unique weather stations: {len(unique_stations)}")

    cell       lat      lon  Height  Azimuth  nearest_station  station_lat  \
0  322_0  51.93255  6.57633    28.3       80             6283      52.0678   
1  322_1  51.93255  6.57633    28.3      190             6283      52.0678   
2  322_2  51.93255  6.57633    28.3      320             6283      52.0678   
3  168_0  52.63140  4.72566    30.5       60             6233      52.4819   
4  168_1  52.63140  4.72566    30.5      180             6233      52.4819   

   station_lon  station_height  distance_km  
0       6.6567           29.07    17.463337  
1       6.6567           29.07    17.463337  
2       6.6567           29.07    17.463337  
3       4.7258           -1.60    16.594507  
4       4.7258           -1.60    16.594507  
Number of unique weather stations: 37


In [4]:
# Load KPI dataset
kpi_df = pd.read_csv(os.path.join(data_dir, "cell_undersampled_2.csv"), parse_dates=["timestamp"])

kpi_df.rename(columns={'cell': 'cell_id'}, inplace=True)

# Convert timestamps to UTC to match Meteo data (assuming KPI data is in Dutch local time)
kpi_df["timestamp"] = kpi_df["timestamp"].dt.tz_localize("Europe/Amsterdam").dt.tz_convert("UTC")
print(kpi_df.head())

                  timestamp  cell_id  bts  antenna  carrier  minRSSI  \
0 2023-09-02 22:00:00+00:00  166_1_0  166        1        0  -106.62   
1 2023-09-02 22:00:00+00:00  168_1_1  168        1        1  -102.35   
2 2023-09-02 22:00:00+00:00  168_2_0  168        2        0   -94.60   
3 2023-09-02 22:00:00+00:00  168_2_1  168        2        1  -100.00   
4 2023-09-02 22:00:00+00:00  174_2_1  174        2        1  -103.89   

   pageSessions     ULvol  sessionSetupDur  sessionDur  blocks  anomaly  
0          37.0  0.086723         8.362069    8.862069       0        0  
1          28.0  0.051203         6.545455    7.090909       0        0  
2          19.0  0.036644         8.866667    9.433333       0        1  
3           6.0  0.033116         0.000000    0.000000       0        1  
4          10.0  0.017862        61.000000   63.500000       0        0  


In [5]:
# Extract the first five characters from the 'cell' column in kpi_df for matching
kpi_df['cell_prefix'] = kpi_df['cell_id'].str[:5]

# Merge the nearest_station column from bts_spatial to kpi_df based on matching cell_prefix and cell
kpi_df = kpi_df.merge(bts_spatial[['nearest_station', 'cell']], left_on='cell_prefix', right_on='cell', how='left')

# Drop the temporary cell_prefix column
kpi_df.drop(columns=['cell_prefix', 'cell'], inplace=True)

In [77]:
print(kpi_df.head(3))
print()
print(kpi_df.tail(1))

                  timestamp  cell_id  bts  antenna  carrier  minRSSI  \
0 2023-09-02 22:00:00+00:00  166_1_0  166        1        0  -106.62   
1 2023-09-02 22:00:00+00:00  168_1_1  168        1        1  -102.35   
2 2023-09-02 22:00:00+00:00  168_2_0  168        2        0   -94.60   

   pageSessions     ULvol  sessionSetupDur  sessionDur  blocks  anomaly  \
0          37.0  0.086723         8.362069    8.862069       0        0   
1          28.0  0.051203         6.545455    7.090909       0        0   
2          19.0  0.036644         8.866667    9.433333       0        1   

   nearest_station  
0             6344  
1             6233  
2             6233  

                       timestamp  cell_id  bts  antenna  carrier  minRSSI  \
395913 2024-09-22 21:30:00+00:00  603_2_1  603        2        1  -104.85   

        pageSessions     ULvol  sessionSetupDur  sessionDur  blocks  anomaly  \
395913          31.0  0.192133         11.52381   12.392857       0        0   

        n

# Download needed meteo datasets

In [7]:
# Define API parameters for downloading only the required data
api_url = "https://api.dataplatform.knmi.nl/open-data/v1/datasets/Actuele10mindataKNMIstations/versions/2/files"
headers = {"Authorization": f"Bearer {api_key}"}

start_date = "2023-09-02T22:00:00+00:00"
end_date = "2024-09-22T21:30:00+00:00"
params = {"begin": start_date, "end": end_date, "orderBy": "created"}

# Convert start_date and end_date to datetime objects
start_datetime = datetime.strptime(start_date, "%Y-%m-%dT%H:%M:%S+00:00")
end_datetime = datetime.strptime(end_date, "%Y-%m-%dT%H:%M:%S+00:00")

samplevar = "tx"

In [86]:
# Fetch files in the date range
response = requests.get(api_url, headers=headers, params=params)
response.raise_for_status()
files = response.json().get("files", [])

# Check if files are returned
if not files:
    print("No files found in the specified date range.")
    exit()

In [96]:
def download_data(max_files=24):
    current_datetime = start_datetime
    downloaded_files_count = 0

    # Loop through each time window (every 30 minutes) until the end_date
    while current_datetime <= end_datetime:
        if downloaded_files_count >= max_files:
            break

        current_datetime_str = current_datetime.strftime("%Y-%m-%dT%H:%M:%S+00:00")
        
        # Define the parameters for the API request
        params = {
            "begin": current_datetime_str,
            "end": (current_datetime + timedelta(minutes=10)).strftime("%Y-%m-%dT%H:%M:%S+00:00"),
            "orderBy": "created"
        }

        # Fetch files for the current time window
        response = requests.get(api_url, headers=headers, params=params)
        response.raise_for_status()
        files_data = response.json()
        files = files_data.get("files", [])

        if not files:
            print(f"No files found for {current_datetime_str}")
        
        # Process and download the files for the current time window
        for file_info in files:
            filename = file_info["filename"]
            timestamp_str = filename.split('_')[-1].replace('.nc', '')
            minute = int(timestamp_str[10:12])  # Extracts minute part
            
            # Download only if minute is '00' or '30'
            if minute == 0 or minute == 30:
        
                # Get the download URL
                file_url = f"{api_url}/{filename}/url"
                response = requests.get(file_url, headers=headers)
                response.raise_for_status()
                download_url = response.json().get("temporaryDownloadUrl")
                
                # Download and save the file
                local_path = os.path.join(save_dir, filename)
                file_data = requests.get(download_url)
                file_data.raise_for_status()

                with open(local_path, "wb") as f:
                    f.write(file_data.content)

                print(f"Downloaded: {filename}")
                downloaded_files_count += 1  # Increment the download count

                # Stop if the download limit is reached
                if downloaded_files_count >= 24:
                    print("Download limit reached (24 files). Stopping.")
                    break


        # Increment the current time window by 30 minutes
        current_datetime += timedelta(minutes=30)

# Start downloading
download_data()

Downloaded: KMDS__OPER_P___10M_OBS_L2_202309022200.nc
Downloaded: KMDS__OPER_P___10M_OBS_L2_202309022230.nc
Downloaded: KMDS__OPER_P___10M_OBS_L2_202309022300.nc
Downloaded: KMDS__OPER_P___10M_OBS_L2_202309022330.nc
Downloaded: KMDS__OPER_P___10M_OBS_L2_202309030000.nc
Downloaded: KMDS__OPER_P___10M_OBS_L2_202309030030.nc
Downloaded: KMDS__OPER_P___10M_OBS_L2_202309030100.nc
Downloaded: KMDS__OPER_P___10M_OBS_L2_202309030130.nc
Downloaded: KMDS__OPER_P___10M_OBS_L2_202309030200.nc
Downloaded: KMDS__OPER_P___10M_OBS_L2_202309030230.nc
Downloaded: KMDS__OPER_P___10M_OBS_L2_202309030300.nc
Downloaded: KMDS__OPER_P___10M_OBS_L2_202309030330.nc
Downloaded: KMDS__OPER_P___10M_OBS_L2_202309030400.nc
Downloaded: KMDS__OPER_P___10M_OBS_L2_202309030430.nc
Downloaded: KMDS__OPER_P___10M_OBS_L2_202309030500.nc
Downloaded: KMDS__OPER_P___10M_OBS_L2_202309030530.nc
Downloaded: KMDS__OPER_P___10M_OBS_L2_202309030600.nc
Downloaded: KMDS__OPER_P___10M_OBS_L2_202309030630.nc
Downloaded: KMDS__OPER_P___1

# Extract sample vars from meteo datasets

In [14]:
def inspect_samplevar(file_path, samplevar='tx', station_var='station'):
    # Open the NetCDF file
    ds = xr.open_dataset(file_path)

    # Extract the samplevar (e.g., 'tx') and station variable
    sample_data = ds[samplevar].values
    station_data = ds[station_var].values

    # Print a snippet of the data (to inspect the values)
    print(f"Sample data from {file_path}:")
    for i in range(min(10, len(sample_data))):  # Limit to the first 10 values or the length of the data
        print(f"Station {station_data[i]}: {samplevar} = {sample_data[i][0]}")  # Adjusting for shape of sample_data

inspect_samplevar(os.path.join(save_dir, "KMDS__OPER_P___10M_OBS_L2_202309022200.nc"))

Sample data from /Users/jakobtjurlik/Library/CloudStorage/OneDrive-Personal/Desktop/Study/MSc_Tilburg/MSc Thesis/code/radio-signal-anomalies/data/meteo_raw/KMDS__OPER_P___10M_OBS_L2_202309022200.nc:
Station 06201: tx = 16.7
Station 06203: tx = 18.2
Station 06204: tx = 17.5
Station 06205: tx = 16.9
Station 06207: tx = 17.5
Station 06208: tx = 17.3
Station 06209: tx = nan
Station 06211: tx = 16.4
Station 06214: tx = 17.2
Station 06215: tx = 14.4
